In [5]:
pip install scrapy

     |████████████████████████████████| 254 kB 5.2 MB/s 
     |████████████████████████████████| 47 kB 4.5 MB/s 
     |████████████████████████████████| 3.2 MB 50.8 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 251 kB 56.9 MB/s 
     |████████████████████████████████| 3.1 MB 44.2 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 3.6 MB 26.7 MB/s 
     |████████████████████████████████| 74 kB 2.9 MB/s 
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7783 sha256=ba0861cce0fe3dd397b0d55039b81a9de78e21125485bd24ea32275fd181f5dd
  Stored in directory: /root/.cache/pip/wheels/ca/44/01/3592ccfbcfaee4ab297c4097e6e9dbe1c7697e3531a39877ab
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11516 sha256=4ab721198fc278a6f949d7f703fa2a72495aa0ee5c23edd544888e73f621da33
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa

In [6]:
pip install crochet

In [7]:
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for
setup()

class QuotesToCsv(scrapy.Spider):
    """scrape first line of  quotes from `wikiquote` by 
    Maynard James Keenan and save to json file"""
    name = "MJKQuotesToCsv"#give any name to our output
    start_urls = [
        'https://en.wikiquote.org/wiki/Maynard_James_Keenan',#url
    ]
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.ExtractFirstLine': 1
        },
        'FEEDS': {
            'quotes.csv': {   #create csv file to save output
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, response):
        """parse data from urls"""        #select the part you want to extract
        for quote in response.css('div.mw-parser-output > ul > li'):
            yield {'quote': quote.extract()}


class ExtractFirstLine(object):
    def process_item(self, item, spider):#for taking quots only
        """text processing"""
        lines = dict(item)["quote"].splitlines()
        first_line = self.__remove_html_tags__(lines[0])

        return {'quote': first_line}

    def __remove_html_tags__(self, text):#to remove html tags
        """remove html tags from string"""
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

@wait_for(10)
def run_spider():
    """run spider with MJKQuotesToCsv"""
    crawler = CrawlerRunner()
    d = crawler.crawl(QuotesToCsv)
    return d

In [11]:
run_spider()
